In [1]:
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 781.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [2]:
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "Get your api key from groq"

In [38]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
ai_msg = llm.invoke("write a poem for my love pizza")
ai_msg
print(ai_msg.content)

Pizza, my love, my heart's desire,
A culinary flame that never tires.
Your crust, so crispy and golden bright,
A beacon in the dark of night.

Your sauce, a symphony of sweet and tang,
A melody that makes my heart sing.
The cheese, oh how it melts and flows,
A river of delight that never grows old.

The toppings, a chorus of flavors and hues,
Each one a note that harmonizes with you.
From savory meats to fresh vegetables,
Together they create a gastronomic spectacle.

Oh, pizza, my love, you're more than just a meal,
You're a passion, a romance that's real.
You satisfy my hunger, soothe my soul,
Make me feel whole, make me feel at home.

So here's to you, dear pizza, my heart's delight,
Forever and always, day and night.
I'll never tire of your charms, your allure,
For you, my love, are the one I desire.


In [4]:
!pip install langchain
!pip install -U langchain-community
!pip install langchain-experimental
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 4.0 MB/s eta 0:00:00


In [5]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_db()

db = SQLDatabase(engine)

In [39]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.run("who are the top fampus artist?")



> Entering new SQLDatabaseChain chain...
who are the top fampus artist?
SQLQuery:Question: Who are the top famous artists?
SQLQuery: SELECT "Name" FROM "Artist" LIMIT 5;
SQLResult: [('AC/DC',), ('Accept',), ('Aerosmith',), ('Alanis Morissette',), ('Alice In Chains',)]
Answer:The top famous artists are AC/DC, Accept, Aerosmith, Alanis Morissette, and Alice In Chains.
> Finished chain.


In [40]:
qns1

'The top famous artists are AC/DC, Accept, Aerosmith, Alanis Morissette, and Alice In Chains.'

In [41]:
qns2 = db_chain.run("How many tracks are there from the 'Rock' genre?")
qns2



> Entering new SQLDatabaseChain chain...
How many tracks are there from the 'Rock' genre?
SQLQuery:SELECT COUNT(*) AS NumberOfTracks
FROM Track
WHERE GenreId = 1;
SQLResult: [(1297,)]
Answer:There are 1297 tracks from the 'Rock' genre.
> Finished chain.


"There are 1297 tracks from the 'Rock' genre."

In [42]:
qns3 = db_chain.run("How much revenue has been generated from all purchases of 'Classical' tracks?")
qns3



> Entering new SQLDatabaseChain chain...
How much revenue has been generated from all purchases of 'Classical' tracks?
SQLQuery:SQLQuery: SELECT SUM(T.UnitPrice * IL.Quantity) AS TotalRevenue FROM Track T INNER JOIN InvoiceLine IL ON T.TrackId = IL.TrackId INNER JOIN Invoice I ON IL.InvoiceId = I.InvoiceId INNER JOIN Genre G ON T.GenreId = G.GenreId WHERE G.Name = 'Classical'
SQLResult: [(40.589999999999996,)]
Answer:The total revenue generated from all purchases of 'Classical' tracks is $40.59.
> Finished chain.


"The total revenue generated from all purchases of 'Classical' tracks is $40.59."

In [43]:
qns4 = db_chain.run("How many albums do we have from the 'Metallica' artist?")
qns4



> Entering new SQLDatabaseChain chain...
How many albums do we have from the 'Metallica' artist?
SQLQuery:SQLQuery: SELECT COUNT(*) FROM Album WHERE ArtistId IN (SELECT ArtistId FROM Artist WHERE Name = 'Metallica')
SQLResult: [(10,)]
Answer:We have 10 albums from the 'Metallica' artist.
> Finished chain.


"We have 10 albums from the 'Metallica' artist."

In [44]:
qns5 = db_chain.run("What is the total revenue generated by the customer 'Frank Harris'?")
qns5



> Entering new SQLDatabaseChain chain...
What is the total revenue generated by the customer 'Frank Harris'?
SQLQuery:SELECT SUM(i.Total) 
FROM Invoice i 
JOIN Customer c ON i.CustomerId = c.CustomerId 
WHERE c.FirstName = 'Frank' AND c.LastName = 'Harris';
SQLResult: [(37.62,)]
Answer:The total revenue generated by the customer 'Frank Harris' is $37.62.
> Finished chain.


"The total revenue generated by the customer 'Frank Harris' is $37.62."

In [12]:
few_shots = [
    {
        'Question': "How many tracks are there from the 'Rock' genre?",
        'SQLQuery': "SELECT COUNT(*) FROM Track JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = 'Rock'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns1
    },
    {
        'Question': "What is the total duration of all tracks by the artist 'AC/DC'?",
        'SQLQuery': """SELECT SUM(Milliseconds / 60000) FROM Track
                       JOIN Album ON Track.AlbumId = Album.AlbumId
                       JOIN Artist ON Album.ArtistId = Artist.ArtistId
                       WHERE Artist.Name = 'AC/DC'""",
        'SQLResult': "Result of the SQL query",
        'Answer': qns2
    },
    {
        'Question': "How much revenue has been generated from all purchases of 'Classical' tracks?",
        'SQLQuery': """SELECT SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) FROM InvoiceLine
                       JOIN Track ON InvoiceLine.TrackId = Track.TrackId
                       JOIN Genre ON Track.GenreId = Genre.GenreId
                       WHERE Genre.Name = 'Classical'""",
        'SQLResult': "Result of the SQL query",
        'Answer': qns3
    },
    {
        'Question': "How many albums do we have from the 'Metallica' artist?",
        'SQLQuery': """SELECT COUNT(*) FROM Album
                       JOIN Artist ON Album.ArtistId = Artist.ArtistId
                       WHERE Artist.Name = 'Metallica'""",
        'SQLResult': "Result of the SQL query",
        'Answer': qns4
    },
    {
        'Question': "What is the total revenue generated by the customer 'Frank Harris'?",
        'SQLQuery': """SELECT SUM(Invoice.Total) FROM Invoice
                       JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
                       WHERE Customer.FirstName = 'Frank' AND Customer.LastName = 'Harris'""",
        'SQLResult': "Result of the SQL query",
        'Answer': qns5
    }
]


In [13]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 3.9 MB/s eta 0:00:00


In [45]:
from langchain.embeddings import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.0 MB/s eta 0:00:

In [46]:
to_vectorize

['How many tracks are there from the \'Rock\' genre? SELECT COUNT(*) FROM Track JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = \'Rock\' Result of the SQL query Question: who are the top famous artist?\nSQLQuery: SELECT T1."Name" FROM "Artist" AS T1 INNER JOIN "Album" AS T2 ON T1."ArtistId" = T2."ArtistId" GROUP BY T1."Name" ORDER BY COUNT(T2."AlbumId") DESC LIMIT 5',
 'What is the total duration of all tracks by the artist \'AC/DC\'? SELECT SUM(Milliseconds / 60000) FROM Track\n                       JOIN Album ON Track.AlbumId = Album.AlbumId\n                       JOIN Artist ON Album.ArtistId = Artist.ArtistId\n                       WHERE Artist.Name = \'AC/DC\' Result of the SQL query Question: What is the total duration of all tracks by the artist \'AC/DC\'?\nSQLQuery: SELECT SUM(T."Milliseconds") \nFROM "Track" AS T \nJOIN "Album" AS A ON T."AlbumId" = A."AlbumId" \nJOIN "Artist" AS Ar ON A."ArtistId" = Ar."ArtistId" \nWHERE Ar."Name" = \'AC/DC\'',
 'How much re

In [47]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [48]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "What are the top 3 most purchased tracks from the 'Pop' genre?"})

[{'Answer': 'Question: who are the top famous artist?\nSQLQuery: SELECT T1."Name" FROM "Artist" AS T1 INNER JOIN "Album" AS T2 ON T1."ArtistId" = T2."ArtistId" GROUP BY T1."Name" ORDER BY COUNT(T2."AlbumId") DESC LIMIT 5',
  'Question': "How many tracks are there from the 'Rock' genre?",
  'SQLQuery': "SELECT COUNT(*) FROM Track JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = 'Rock'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'Question: who are the top famous artist?\nSQLQuery: SELECT T1."Name" FROM "Artist" AS T1 INNER JOIN "Album" AS T2 ON T1."ArtistId" = T2."ArtistId" GROUP BY T1."Name" ORDER BY COUNT(T2."AlbumId") DESC LIMIT 5',
  'Question': "How many tracks are there from the 'Rock' genre?",
  'SQLQuery': "SELECT COUNT(*) FROM Track JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = 'Rock'",
  'SQLResult': 'Result of the SQL query'}]

In [19]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [49]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [50]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [51]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [52]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [53]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [54]:
new_chain("How many tracks are there from the 'Rock' genre?")



> Entering new SQLDatabaseChain chain...
How many tracks are there from the 'Rock' genre?
SQLQuery:SELECT COUNT(*) FROM Track JOIN Genre ON Track.GenreId = Genre.GenreId WHERE Genre.Name = 'Rock'
SQLResult: [(1297,)]
Answer:There are 1297 tracks from the 'Rock' genre.
> Finished chain.


{'query': "How many tracks are there from the 'Rock' genre?",
 'result': "There are 1297 tracks from the 'Rock' genre."}

In [55]:
new_chain("What is the total duration of all tracks by the artist 'AC/DC'?")



> Entering new SQLDatabaseChain chain...
What is the total duration of all tracks by the artist 'AC/DC'?
SQLQuery:SELECT SUM(T."Milliseconds") / 60000 
FROM "Track" AS T 
JOIN "Album" AS A ON T."AlbumId" = A."AlbumId" 
JOIN "Artist" AS Ar ON A."ArtistId" = Ar."ArtistId" 
WHERE Ar."Name" = 'AC/DC'
SQLResult: [(80,)]
Answer:The total duration of all tracks by the artist 'AC/DC' is 80 minutes.
> Finished chain.


{'query': "What is the total duration of all tracks by the artist 'AC/DC'?",
 'result': "The total duration of all tracks by the artist 'AC/DC' is 80 minutes."}

In [56]:
new_chain("How much revenue has been generated from all purchases of 'Classical' tracks?")



> Entering new SQLDatabaseChain chain...
How much revenue has been generated from all purchases of 'Classical' tracks?
SQLQuery:SELECT SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) FROM InvoiceLine
                       JOIN Track ON InvoiceLine.TrackId = Track.TrackId
                       JOIN Genre ON Track.GenreId = Genre.GenreId
                       WHERE Genre.Name = 'Classical'
SQLResult: [(40.589999999999996,)]
Answer:The total revenue generated from all purchases of 'Classical' tracks is approximately $40.59.
> Finished chain.


{'query': "How much revenue has been generated from all purchases of 'Classical' tracks?",
 'result': "The total revenue generated from all purchases of 'Classical' tracks is approximately $40.59."}

In [57]:
new_chain("What is the total revenue generated by the customer 'Frank Harris'?")



> Entering new SQLDatabaseChain chain...
What is the total revenue generated by the customer 'Frank Harris'?
SQLQuery:SELECT SUM(I."Total") FROM "Invoice" I INNER JOIN "Customer" C ON I."CustomerId" = C."CustomerId" WHERE C."FirstName" = 'Frank' AND C."LastName" = 'Harris'
SQLResult: [(37.62,)]
Answer:The total revenue generated by the customer 'Frank Harris' is $37.62.
> Finished chain.


{'query': "What is the total revenue generated by the customer 'Frank Harris'?",
 'result': "The total revenue generated by the customer 'Frank Harris' is $37.62."}

In [58]:
new_chain("How many albums do we have from the 'Metallica' artist?")



> Entering new SQLDatabaseChain chain...
How many albums do we have from the 'Metallica' artist?
SQLQuery:SELECT COUNT(*) FROM Album
                       JOIN Artist ON Album.ArtistId = Artist.ArtistId
                       WHERE Artist.Name = 'Metallica'
SQLResult: [(10,)]
Answer:We have 10 albums from the 'Metallica' artist.
> Finished chain.


{'query': "How many albums do we have from the 'Metallica' artist?",
 'result': "We have 10 albums from the 'Metallica' artist."}